In [2]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [4]:
f = open("ashtanga_urls", "rb")
ashtanga_urls = pickle.load(f)
f.close()

In [5]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Ashtanga"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [6]:
ashtanga_classes = [get_poses(url) for url in ashtanga_urls]

In [8]:
df = pd.DataFrame(ashtanga_classes)
df

,Title,Poses,Class Type
0,←KCADY Week Ashtanga,"[Sun Salutation A First Half, Sun Salutation A...",Ashtanga
1,←Master Flow: Ashtanga Yoga Remix,"[Easy Pose, Introduce Breath Section, Downward...",Ashtanga
2,←Beach Flow - Ashtanga flow,"[Child Pose, Cobra Pose, Wide Child Pose, Sphi...",Ashtanga
3,←Peace and Power,"[Warmup, Sun Salutation A Second Half, Key Pos...",Ashtanga
4,←KCADY Season Week Ashtanga,"[Sun Salutation A First Half, Sun Salutation A...",Ashtanga
...,...,...,...
1394,←NATARAJASANA (Vũ công),"[Fish Pose Blocks Knees Positions, Cat Cow Pos...",Ashtanga
1395,←Série Primaire débutant,"[Sun Salutation A, Sun Salutation B, Mountain ...",Ashtanga
1396,←HANDSTAND,"[Hero Pose, Hero Pose Variation Hands Raised, ...",Ashtanga
1397,←Hip Opening Sequence,"[Hand to Big Toe Pose, Standing Forward Bend H...",Ashtanga


In [9]:
f = open("ashtanga_df", "wb")
pickle.dump(df, f)
f.close()